# ETL

### Cargo el dataset correspondiente desde la carpeta /sources para posteriormente analizarlo, limpiarlo y guardarlo en la carpeta /Data

In [1]:
# Importamos las librerias que vamos a utilizar
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Acceso a el cvs y se pasa a un dataframe 
data = pd.read_csv('../sources/ref_temperatura_lluvia_jun2022_jun2024.csv')

In [3]:
# Visualización de manera general
data.head()

,latitude,longitude,elevation,utc_offset_seconds,timezone,timezone_abbreviation
0,52.54833,13.407822,38.0,-14400,America/New_York,EDT
1,time,temperature_2m (°C),rain (mm),is_day (),NaN,NaN
2,2022-06-30T00:00,17.7,0.10,1,NaN,NaN
3,2022-06-30T01:00,18.2,0.00,1,NaN,NaN
4,2022-06-30T02:00,19.4,0.00,1,NaN,NaN


## Identifico y separo las diferentes partes del dataset

- Primeras filas contienen información geográfica y de zona horaria.

- Filas posteriores contienen datos metereológicos.

### Limpieza de la sección meteorológica y geográfica.

In [4]:
# Identifico el índice dónde empiezan los datos meteorológicos
weather_start_idx = data[data['latitude'] == 'time'].index[0]

In [5]:
# Separo datos geográficos y de zona horaria
geo_df = data.iloc[:weather_start_idx].copy()

In [6]:
# Separo datos meteorológicos
weather_df = data.iloc[weather_start_idx+1:].copy()

In [7]:
# Restablezco los índices de ambos df
geo_df.reset_index(drop=True, inplace=True)
weather_df.reset_index(drop=True, inplace=True)

In [8]:
# Convierto columnas a tipos de datos nmericos flotantes
geo_df['latitude'] = geo_df['latitude'].astype(float)
geo_df['longitude'] = geo_df['longitude'].astype(float)
geo_df['elevation'] = geo_df['elevation'].astype(float)
geo_df['utc_offset_seconds'] = geo_df['utc_offset_seconds'].astype(int)

In [9]:
# Renombro las columnas
weather_df.columns = ['time', 'temperature_2m (°C)', 'rain (mm)', 'is_day ()', 'extra_1', 'extra_2']

In [10]:
# Elimino columnas innecesarias
weather_df = weather_df[['time', 'temperature_2m (°C)', 'rain (mm)', 'is_day ()']]

In [11]:
# Convierto la columna de tiempo al formato datetime
weather_df['time'] = pd.to_datetime(weather_df['time'])

In [12]:
# Guarda los dataframes en la carpeta Data
geo_df.to_parquet('../Data/datos_geograficos_limpios.parquet', index=False)
weather_df.to_parquet('../Data/datos_meteorologicos_limpios.parquet', index=False)